In [ ]:
!pip install fredapi > /dev/null

In [ ]:
# Import necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time
from fredapi import Fred
from kaggle_secrets import UserSecretsClient

# Set plot style and options
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 500)

# Initialize Fred API with secret key
secrets = UserSecretsClient()
fred_key = secrets.get_secret('fred-api')
fred = Fred(api_key=fred_key)

# Function to fetch and plot S&P 500 data
def fetch_sp500():
    sp500 = fred.get_series('SP500')
    plt.figure(figsize=(10, 5))
    sp500.plot(title='S&P 500', lw=2)
    plt.show()

# Function to fetch and clean unemployment rate data
def fetch_unemployment_data():
    unemp_df = fred.search('unemployment rate state', filter=('frequency', 'Monthly'))
    unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
    unemp_df = unemp_df.loc[unemp_df['title'].str.contains('Unemployment Rate')]

    all_results = []
    for myid in unemp_df.index:
        results = fred.get_series(myid).to_frame(name=myid)
        all_results.append(results)
        time.sleep(0.1)  # To avoid request limits

    uemp_results = pd.concat(all_results, axis=1)
    cols_to_drop = [col for col in uemp_results if len(uemp_results[col]) > 4]
    uemp_results = uemp_results.drop(columns=cols_to_drop)
    uemp_states = uemp_results.dropna()
    
    id_to_state = unemp_df['title'].str.replace('Unemployment Rate in ', '').to_dict()
    uemp_states.columns = [id_to_state[c] for c in uemp_states.columns]

    return uemp_states

# Function to plot state unemployment rates
def plot_unemployment_rates(uemp_states):
    fig = px.line(uemp_states, title='Unemployment Rate by State')
    fig.show()

    latest_data = uemp_states.loc['2020-05-01'].sort_values()
    plt.figure(figsize=(8, 12))
    latest_data.plot(kind='barh', edgecolor='black', title='Unemployment Rate by State, May 2020')
    plt.xlabel('% Unemployed')
    plt.show()

# Function to fetch and clean labor force participation rate data
def fetch_participation_data():
    part_df = fred.search('participation rate state', filter=('frequency', 'Monthly'))
    part_df = part_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
    part_id_to_state = part_df['title'].str.replace('Labor Force Participation Rate for ', '').to_dict()

    all_results = []
    for myid in part_df.index:
        results = fred.get_series(myid).to_frame(name=myid)
        all_results.append(results)
        time.sleep(0.1)  # To avoid request limits

    part_states = pd.concat(all_results, axis=1)
    part_states.columns = [part_id_to_state[c] for c in part_states.columns]
    
    return part_states

# Function to plot comparison of unemployment and participation rates
def plot_comparison(uemp_states, part_states):
    uemp_states = uemp_states.rename(columns={'the District of Columbia': 'District Of Columbia'})
    
    fig, axs = plt.subplots(10, 5, figsize=(30, 30), sharex=True)
    axs = axs.flatten()

    for i, state in enumerate(uemp_states.columns):
        if state in ["District Of Columbia", "Puerto Rico"]:
            continue
        ax2 = axs[i].twinx()
        uemp_states.query('index >= 2020 and index < 2022')[state].plot(ax=axs[i], label='Unemployment')
        part_states.query('index >= 2020 and index < 2022')[state].plot(ax=ax2, label='Participation', color='orange')
        ax2.grid(False)
        axs[i].set_title(state)
    
    plt.tight_layout()
    plt.show()

# Function to plot detailed unemployment and participation for a specific state
def plot_state_comparison(state, uemp_states, part_states):
    uemp_states = uemp_states.asfreq('MS')
    fig, ax = plt.subplots(figsize=(10, 5), sharex=True)
    ax2 = ax.twinx()
    
    uemp_states.query('index >= 2020 and index < 2022')[state].plot(ax=ax, label='Unemployment')
    part_states.query('index >= 2020 and index < 2022')[state].plot(ax=ax2, label='Participation', color='orange')
    
    ax2.grid(False)
    ax.set_title(state)
    fig.legend(labels=['Unemployment', 'Participation'])
    plt.show()

# Main execution
if __name__ == "__main__":
    fetch_sp500()
    uemp_states = fetch_unemployment_data()
    plot_unemployment_rates(uemp_states)
    part_states = fetch_participation_data()
    plot_comparison(uemp_states, part_states)
    plot_state_comparison('California', uemp_states, part_states)